<a href="https://colab.research.google.com/github/mingyaw/TradingStock/blob/main/USTradingStock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

美股前300大列表


In [ ]:
import pandas as pd
import requests
from io import StringIO


# def us_stork_list():
try:
    df_list = []
    # 加入headers
    headers = {
        "user-agent": "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.157 Safari/537.36"
    }
    # 對網站進行requests，並加入指定的headers一同請求
    for i in range(0, 300, 100):
        url = 'https://finance.yahoo.com/screener/predefined/most_actives?count=100&offset=%d' % i
        html_data = requests.get(url, headers=headers)
        # 使用pandas的read_html處理表格式
        data = pd.read_html(StringIO(html_data.text))[0]
        print(data.index)
        df_list.append(data)

    # stk_list
    df = pd.concat(df_list)
    df.to_csv('TradingStock/us_stock_list.csv')

except SystemExit:
    print('Its OK')



取單支股票10年每日股價


In [ ]:
import pandas as pd
import requests
import yfinance as yf
import numpy as np
from prophet import Prophet
import matplotlib.pyplot as plt

start_date = '2014-01-01'
end_date = '2024-12-31'

stock_list = pd.read_csv('TradingStock/us_stock_list.csv')

for i in range(stock_list.shape[0]-1):
     c_time = stock_list.index[i]
     stock_no = stock_list.loc[c_time, 'Symbol']
     df = yf.download(stock_no,start=start_date,end=end_date)
     df.to_csv('TradingStock/US_DATA/'+stock_no +'_2014_2024.csv')

美股回測

In [ ]:
!pip install backtrader
import backtrader as bt
import pandas as pd
import numpy as np
import datetime
from copy import deepcopy

class SmaCross(bt.SignalStrategy):
    def __init__(self):
        sma1, sma2 = bt.ind.SMA(period=10), bt.ind.SMA(period=30)
        crossover = bt.ind.CrossOver(sma1, sma2)
        self.signal_add(bt.SIGNAL_LONG, crossover)

class BollStrategy(bt.SignalStrategy):
    def __init__(self):
        sma1, sma2 = bt.ind.SMA(period=10), bt.ind.SMA(period=30)
        crossover = bt.ind.CrossOver(sma1, sma2)
        self.signal_add(bt.SIGNAL_LONG, crossover)

class MACDStrategy(bt.Strategy):
  params=(('p1',12),('p2',26),('p3',9),)
  def __init__(self):
    self.order = None
    #取得MACD
    self.macdhist = bt.ind.MACDHisto(self.data,
    period_me1=self.p.p1,
    period_me2=self.p.p2,
    period_signal=self.p.p3)

  def next(self):
    if not self.position:
      # 得到當前的帳戶價值
      total_value = self.broker.getvalue

      #1手=100股，滿倉買入
      # ss=int((total_value/100)/self.datas[0].close[0])*100

      #當MACD柱大於0（紅柱）且無持倉時滿倉買入
      if self.macdhist > 0:
        self.order=self.buy(size=100)
    else:
      #當MACD柱小於0（綠柱）且持倉時全部清倉
      if self.macdhist < 0:
        self.close

if __name__ == '__main__':
  cerebro = bt.Cerebro()
  # 開始資金
  print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

  dataframe = pd.read_csv('TradingStock/US_DATA/AAPL_2014_2024.csv', index_col=0, parse_dates=True)
  dataframe['openinterest'] = 0
  data = bt.feeds.PandasData(dataname=dataframe,
                          fromdate = datetime.datetime(2014, 1, 1),
                          todate = datetime.datetime(2024, 12, 31)
                          )
  cerebro.adddata(data)
  cerebro.addstrategy(MACDStrategy)
  cerebro.broker.set_cash(cash=10000)
  cerebro.run()
  # cerebro.plot()
  # 回測完成資金
  print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())